In [107]:
from selenium import webdriver
import os
import pandas as pd
import time

In [ ]:
driver = webdriver.Chrome(os.path.join(os.getcwd(), 'chromedriver'))


def geturls(url):
    driver.get(url)
    load_more = driver.find_element_by_class_name("load-more ")
    for i in range(20):
        load_more.click()
        time.sleep(1)
    contents = driver.find_elements_by_class_name("post-block__title__link")
    urls = []
    for elem in contents:
        urls.append(elem.get_attribute("href"))
    urls = set(urls)
    driver.close()
    return urls


urls = geturls('https://techcrunch.com/')

In [109]:
len(urls)

80

In [70]:
driver = webdriver.Chrome(os.path.join(os.getcwd(), 'chromedriver'))
data = []
for url in urls:
    driver.get(url)
    heading = driver.find_element_by_class_name('article__title').text
    body = driver.find_element_by_class_name('article-content').text
    tagBlock = driver.find_element_by_class_name('article__tags')
    tags = tagBlock.find_elements_by_class_name('menu__item')
    tagsString = ''
    first = True
    for tag in tags:
        if '(' not in tag.text and '-' not in tag.text and 'TC' not in tag.text and '&amp;' not in tag.text:
            if first:
                first = False
                tagsString = tag.text
            else:
                tagsString += ',' + tag.text
    row = (heading, body, tagsString.strip())
    data.append(row)
driver.close()

df = pd.DataFrame(data)

In [71]:
df.columns = ['Title', 'Body', 'Tags']
df.head()

,Title,Body,Tags
0,3DEN raises $2M to create pay-as-you-go urban ...,3DEN is building spaces for what it calls the ...,"Apps,Funding,Startups,3DEN,coworking"
1,How to decarbonize America — and the world,Ramez Naam\nContributor\nMore posts by this co...,"Column,GreenTech,agriculture,agriculture tech,..."
2,Daily Crunch: Amazon scraps HQ2 plans in NYC,The Daily Crunch is TechCrunch’s roundup of ou...,"eCommerce,Policy,Daily Crunch,hq2"
3,Samsung is preparing to launch a sports smartw...,Samsung’s newest product launch happens next w...,"Gadgets,AirPods,computing,Samsung,samsung gala..."
4,"Peloton pedals toward an IPO, self-driving is ...","Hello and welcome back to Equity, TechCrunch’s...","Podcasts,Venture Capital,Equity podcast"


In [72]:
df[['Body']]

,Body
0,3DEN is building spaces for what it calls the ...
1,Ramez Naam\nContributor\nMore posts by this co...
2,The Daily Crunch is TechCrunch’s roundup of ou...
3,Samsung’s newest product launch happens next w...
4,"Hello and welcome back to Equity, TechCrunch’s..."
5,"Rivian, the electric automaker that debuted it..."
6,"ChargedUp, a U.K. startup that offers a mobile..."
7,Spotify doubled down on podcasts last week wit...
8,"Three cities, three dead urban unicorn renewal..."
9,"If you’re a Coinbase user, you may have seen s..."


In [73]:
df = df[df['Tags'] != '']

In [74]:
df.shape

(20, 3)

In [75]:
df_final = pd.concat([df[['Title','Body']], df['Tags'].str.split(',', expand=True)], axis=1)

In [76]:
df_final.head(10)

,Title,Body,0,1,2,3,4,5,6,7
0,3DEN raises $2M to create pay-as-you-go urban ...,3DEN is building spaces for what it calls the ...,Apps,Funding,Startups,3DEN,coworking,None,None,None
1,How to decarbonize America — and the world,Ramez Naam\nContributor\nMore posts by this co...,Column,GreenTech,agriculture,agriculture tech,Congress,democrats,electric vehicles,None
2,Daily Crunch: Amazon scraps HQ2 plans in NYC,The Daily Crunch is TechCrunch’s roundup of ou...,eCommerce,Policy,Daily Crunch,hq2,None,None,None,None
3,Samsung is preparing to launch a sports smartw...,Samsung’s newest product launch happens next w...,Gadgets,AirPods,computing,Samsung,samsung galaxy,smartwatch,None,None
4,"Peloton pedals toward an IPO, self-driving is ...","Hello and welcome back to Equity, TechCrunch’s...",Podcasts,Venture Capital,Equity podcast,None,None,None,None,None
5,Amazon leads $700M round in electric automaker...,"Rivian, the electric automaker that debuted it...",Automotive,Funding,Transportation,electric vehicles,michigan,Mitsubishi,Rivian,None
6,ChargedUp picks up £1.2M seed to grow its mobi...,"ChargedUp, a U.K. startup that offers a mobile...",Europe,Mobile,Startups,Batteries,ChargedUp,None,None,None
7,Spotify says it paid $340M to buy Gimlet and A...,Spotify doubled down on podcasts last week wit...,Media,Podcasts,None,None,None,None,None,None
8,Urban unicorn renewal,"Three cities, three dead urban unicorn renewal...",Government,Policy,Economic Development,Foxconn,hq2,None,None,None
9,Coinbase users can now withdraw Bitcoin SV fol...,"If you’re a Coinbase user, you may have seen s...",Blockchain,Cryptocurrency,Distributed Ledger,BCH,Bitcoin,Bitcoin ABC,None,None


In [77]:
dfT = df_final[[0, 1, 2, 3, 4, 5, 6, 7]]
dfT.head()

,0,1,2,3,4,5,6,7
0,Apps,Funding,Startups,3DEN,coworking,None,None,None
1,Column,GreenTech,agriculture,agriculture tech,Congress,democrats,electric vehicles,None
2,eCommerce,Policy,Daily Crunch,hq2,None,None,None,None
3,Gadgets,AirPods,computing,Samsung,samsung galaxy,smartwatch,None,None
4,Podcasts,Venture Capital,Equity podcast,None,None,None,None,None


In [44]:
dfBA = df_final[['Title','Body']]
dfBA.columns = [0,1]
dfBA.head()

,0,1
0,3DEN raises $2M to create pay-as-you-go urban ...,3DEN is building spaces for what it calls the ...
1,How to decarbonize America — and the world,Ramez Naam\nContributor\nMore posts by this co...
2,Daily Crunch: Amazon scraps HQ2 plans in NYC,The Daily Crunch is TechCrunch’s roundup of ou...
3,Samsung is preparing to launch a sports smartw...,Samsung’s newest product launch happens next w...
4,"Peloton pedals toward an IPO, self-driving is ...","Hello and welcome back to Equity, TechCrunch’s..."


In [78]:
df=pd.concat([dfBA,dfT],axis=1)
df.columns = [0,1,2,3,4,5,6,7,8,9]
df.head()

,0,1,2,3,4,5,6,7,8,9
0,3DEN raises $2M to create pay-as-you-go urban ...,3DEN is building spaces for what it calls the ...,Apps,Funding,Startups,3DEN,coworking,None,None,None
1,How to decarbonize America — and the world,Ramez Naam\nContributor\nMore posts by this co...,Column,GreenTech,agriculture,agriculture tech,Congress,democrats,electric vehicles,None
2,Daily Crunch: Amazon scraps HQ2 plans in NYC,The Daily Crunch is TechCrunch’s roundup of ou...,eCommerce,Policy,Daily Crunch,hq2,None,None,None,None
3,Samsung is preparing to launch a sports smartw...,Samsung’s newest product launch happens next w...,Gadgets,AirPods,computing,Samsung,samsung galaxy,smartwatch,None,None
4,"Peloton pedals toward an IPO, self-driving is ...","Hello and welcome back to Equity, TechCrunch’s...",Podcasts,Venture Capital,Equity podcast,None,None,None,None,None


In [93]:
from collections import Counter
okList=[]
for cl in dfT.columns:
    for n in dfT[cl]:
        okList.append(n)
# okList=list(set(okList))
okList = list(filter(None, okList))
counter = Counter(okList)
print(counter)
top_ten = [tag for tag,cnt in counter.most_common(10)]
print(top_ten)

newDF=pd.DataFrame(columns=top_ten)
newDF.head()

Counter({'Transportation': 4, 'Apps': 3, 'Column': 2, 'Podcasts': 2, 'Europe': 2, 'Real Estate': 2, 'Funding': 2, 'Policy': 2, 'Samsung': 2, 'Startups': 2, 'hq2': 2, 'electric vehicles': 2, 'eCommerce': 1, 'Gadgets': 1, 'Automotive': 1, 'Media': 1, 'Government': 1, 'Blockchain': 1, 'Enterprise': 1, 'Hardware': 1, 'Asia': 1, 'Earnings': 1, 'david stern': 1, 'GreenTech': 1, 'AirPods': 1, 'Venture Capital': 1, 'Mobile': 1, 'Cryptocurrency': 1, 'drones': 1, 'Privacy': 1, 'Personnel': 1, 'energy': 1, 'africa': 1, 'agriculture': 1, 'Daily Crunch': 1, 'computing': 1, 'Equity podcast': 1, 'Economic Development': 1, 'Distributed Ledger': 1, 'boston': 1, 'Dubai': 1, 'Security': 1, 'earnings': 1, 'national basketball association': 1, 'broadband': 1, '3DEN': 1, 'agriculture tech': 1, 'Batteries': 1, 'Foxconn': 1, 'BCH': 1, 'cambridge': 1, 'Social': 1, 'carsharing': 1, 'NBA': 1, 'ceo': 1, 'coworking': 1, 'Congress': 1, 'samsung galaxy': 1, 'michigan': 1, 'ChargedUp': 1, 'Bitcoin': 1, 'ge': 1, 'api'

,Transportation,Apps,Column,Podcasts,Europe,Real Estate,Funding,Policy,Samsung,Startups


In [94]:
for x in range(dfT.count()[0]):
    someDict={}
    for d in okList:
        rowdata=list(dfT.iloc[x])
        if d in rowdata:
            someDict[d]=1
        else:
            someDict[d]=0
    newDF=newDF.append(someDict,ignore_index=True)
    
dfBA.index=range(dfBA.count()[0])
df = pd.DataFrame()
df=dfBA.join(newDF)
df.head()

,0,1,Transportation,Apps,Column,Podcasts,Europe,Real Estate,Funding,Policy,...,hq2,journalist,law,layoffs,michigan,national basketball association,renewable energy,samsung galaxy,smartwatch,solar panel
0,3DEN raises $2M to create pay-as-you-go urban ...,3DEN is building spaces for what it calls the ...,0,1,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,How to decarbonize America — and the world,Ramez Naam\nContributor\nMore posts by this co...,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Daily Crunch: Amazon scraps HQ2 plans in NYC,The Daily Crunch is TechCrunch’s roundup of ou...,0,0,0,0,0,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Samsung is preparing to launch a sports smartw...,Samsung’s newest product launch happens next w...,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,"Peloton pedals toward an IPO, self-driving is ...","Hello and welcome back to Equity, TechCrunch’s...",0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
df_temp = df.drop(columns = [0, 1], axis = 1)
final_columns = ['Title', 'Body'] + list(df_temp.columns)
df.columns = final_columns
df_final = df.iloc[:,0:12] 
df_final.head(20)

,Title,Body,Transportation,Apps,Column,Podcasts,Europe,Real Estate,Funding,Policy,Samsung,Startups
0,3DEN raises $2M to create pay-as-you-go urban ...,3DEN is building spaces for what it calls the ...,0,1,0,0,0,0,1,0,0,1
1,How to decarbonize America — and the world,Ramez Naam\nContributor\nMore posts by this co...,0,0,1,0,0,0,0,0,0,0
2,Daily Crunch: Amazon scraps HQ2 plans in NYC,The Daily Crunch is TechCrunch’s roundup of ou...,0,0,0,0,0,0,0,1,0,0
3,Samsung is preparing to launch a sports smartw...,Samsung’s newest product launch happens next w...,0,0,0,0,0,0,0,0,1,0
4,"Peloton pedals toward an IPO, self-driving is ...","Hello and welcome back to Equity, TechCrunch’s...",0,0,0,1,0,0,0,0,0,0
5,Amazon leads $700M round in electric automaker...,"Rivian, the electric automaker that debuted it...",1,0,0,0,0,0,1,0,0,0
6,ChargedUp picks up £1.2M seed to grow its mobi...,"ChargedUp, a U.K. startup that offers a mobile...",0,0,0,0,1,0,0,0,0,1
7,Spotify says it paid $340M to buy Gimlet and A...,Spotify doubled down on podcasts last week wit...,0,0,0,1,0,0,0,0,0,0
8,Urban unicorn renewal,"Three cities, three dead urban unicorn renewal...",0,0,0,0,0,0,0,1,0,0
9,Coinbase users can now withdraw Bitcoin SV fol...,"If you’re a Coinbase user, you may have seen s...",0,0,0,0,0,0,0,0,0,0


In [97]:
df_final1 = df_final.iloc[:,2:]
df_final1 = df_final[(df_final.iloc[:,2:].T != 0).any()]
df_final1.shape

(17, 12)

In [99]:
df_final1

,Title,Body,Transportation,Apps,Column,Podcasts,Europe,Real Estate,Funding,Policy,Samsung,Startups
0,3DEN raises $2M to create pay-as-you-go urban ...,3DEN is building spaces for what it calls the ...,0,1,0,0,0,0,1,0,0,1
1,How to decarbonize America — and the world,Ramez Naam\nContributor\nMore posts by this co...,0,0,1,0,0,0,0,0,0,0
2,Daily Crunch: Amazon scraps HQ2 plans in NYC,The Daily Crunch is TechCrunch’s roundup of ou...,0,0,0,0,0,0,0,1,0,0
3,Samsung is preparing to launch a sports smartw...,Samsung’s newest product launch happens next w...,0,0,0,0,0,0,0,0,1,0
4,"Peloton pedals toward an IPO, self-driving is ...","Hello and welcome back to Equity, TechCrunch’s...",0,0,0,1,0,0,0,0,0,0
5,Amazon leads $700M round in electric automaker...,"Rivian, the electric automaker that debuted it...",1,0,0,0,0,0,1,0,0,0
6,ChargedUp picks up £1.2M seed to grow its mobi...,"ChargedUp, a U.K. startup that offers a mobile...",0,0,0,0,1,0,0,0,0,1
7,Spotify says it paid $340M to buy Gimlet and A...,Spotify doubled down on podcasts last week wit...,0,0,0,1,0,0,0,0,0,0
8,Urban unicorn renewal,"Three cities, three dead urban unicorn renewal...",0,0,0,0,0,0,0,1,0,0
10,"As GE and Amazon move on, Google expands prese...",NYC and Boston were handed huge setbacks this ...,0,0,0,0,0,1,0,0,0,0


In [101]:
# df.to_excel('data.xlsx', index = False)
from pandas import ExcelWriter

writer = ExcelWriter('data_top_10.xlsx')
df_final1.to_excel(writer,'Sheet1')
writer.save()